In [3]:
#IMPORT DELLE LIBRERIE
import pandas as pd
import numpy as np
import plotly.express as px

In [9]:
#IMPORT DEI DATI
df= pd.read_csv('https://github.com/LilloBarilotto/LabInternet2022/blob/main/Report2_Test_Ping_Plots/Data_txt/100Switch.txt?raw=true',delimiter=' ')
df

,Size,RttMin,RttAvg,RttMax
0,16,0.536,0.588,0.619
1,116,0.555,0.641,0.696
2,216,0.631,0.710,0.738
3,316,0.690,0.769,0.795
4,416,0.775,0.824,0.857
...,...,...,...,...
126,9600,3.205,3.427,3.531
127,9700,3.379,3.492,3.523
128,9800,3.342,3.481,3.540
129,9900,3.264,3.478,3.536


In [10]:
#Definizioni per RTT

maxSize = 1472 #Max Size deriva da 1500=MTU - 20 byte IPHeader - 8byte ICMPHeader
L2size  = 1538 #total size with max payload =1500
headers = 58  #ICMP + IP + Ethernet

def getLenght(s):
    mod = (s%maxSize)
    a = (s // maxSize) * L2size
    if (mod != 0):
        a = a + mod + getPadding(mod) + headers
    return a

def getPadding(x):
    if (x < 18): return 18-x
    return 0

def getRttTheory(d, C):
    res= 2*d*8*(10**3)/(C)
    return res

def getRttTheory_Switch(d, C):
    if(d<=1538):
        result = 2*getRttTheory(d,C)
    else:
        result = getRttTheory(d,C) + getRttTheory(1538,C) 
    
    return result

    
getLenght  = np.vectorize(getLenght)
getPadding = np.vectorize(getPadding)
getRttTheory_Switch= np.vectorize(getRttTheory_Switch)

In [11]:
#CALCOLO VALORI EFFETTIVI

#Togli il commento a una delle due per settare la capacità
#Cap=10*10**6
Cap=100*10**6

lenght = getLenght(df['Size'])
RttTheory = getRttTheory(lenght, Cap)

#se l'ultimo serve nel caso si voglia stampare anche quello, togli #
RttTheorySwitch = getRttTheory_Switch(lenght, Cap)

In [17]:
#Stampa RTT

fig = px.line(df, x=lenght, y=['RttMin', 'RttAvg', 'RttMax', RttTheory, RttTheorySwitch],
                markers=True ,labels=dict(x="Data [bytes]", value="RTT [ms]", variable="Tipologia di RTT"),
                width=1400, height= 900)

newnamesFig = {'RttMin':'RttMin', 'RttAvg':'RttAvg', 'RttMax':'RttMax', 'wide_variable_3': 'RttTheory','wide_variable_4': 'RttTheorySwitch(Store&Forward)'}
fig.for_each_trace(lambda t: t.update(name = newnamesFig[t.name], legendgroup = newnamesFig[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnamesFig[t.name])
                                     )
                  )

fig.update_layout(legend=dict(  yanchor="top", y=0.99, xanchor="left", x=0.01 ))
fig.show()

In [16]:
#Definizioni per la capacità
def getCapacity_sec(d, rtt):
    x=  2*8*d/rtt/1000
    return x

def getCapacity_sec_TheorySwitch(d, rtt):
    if(d<=1538):
        return 2*getCapacity_sec(d, rtt)
    if(Cap==10*10**6):                      # 2.4 nel caso di 10Mbit/s, altrimenti 0.24 per 100Mbit/s
        return 2*8*d/(rtt -2.4)/1000
    return 2*8*d/(rtt -0.24)/1000

getCapacity_sec_TheorySwitch= np.vectorize(getCapacity_sec_TheorySwitch)

In [20]:
#Calcolo delle capacità
Speed= getCapacity_sec(lenght, df['RttMin'])
SpeedTheory = getCapacity_sec(lenght, RttTheory)

SpeedTheorySwitch = getCapacity_sec_TheorySwitch(lenght, RttTheorySwitch)
SpeedSwitch= getCapacity_sec_TheorySwitch(lenght, df['RttMin'])

In [18]:
#STAMPA CAPACITA'

fig2 = px.line(df, x=lenght, y=[Speed, SpeedTheory, SpeedSwitch, SpeedTheorySwitch],
            labels=dict(x="Data [bytes]", value="Capacity [Mbit/s]", variable="Types of Capacity"))

newnamesFig2 = {'wide_variable_0':'Capacity(RttMin)', 'wide_variable_1':'Capacity(RttTheory)', 'wide_variable_2':'Capacity(RttMinSwitch)', 'wide_variable_3':'Capacity(RttTheorySwitch)'}
fig2.for_each_trace(lambda t: t.update(name = newnamesFig2[t.name],
                                      legendgroup = newnamesFig2[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnamesFig2[t.name])
                                     )
                  )

fig2.update_layout(legend=dict( yanchor="top", y=0.90, xanchor="left", x=0.01))
fig2.show()

NameError: name 'Speed' is not defined